### SageMaker fine tune baichuan

#### 准备
1. 升级boto3, sagemaker python sdk  
2. 准备requirements.txt

In [1]:
!pip install --upgrade boto3
!pip install --upgrade sagemaker

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 116.9 MB/s eta 0:00:0000:010:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.29.114
    Uninstalling botocore-1.29.114:
      Successfully uninstalled botocore-1.29.114
  Attempting uninstall: boto3
    Found existing installation: boto3 1.26.114
    Uninstalling boto3-1.26.114:
      Successfully uninstalled boto3-1.26.114
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.27.114 requires botocore==1.29.114, but you have botocore 1.31.1 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 853.5/853.5 kB 16.4

In [5]:
import boto3
import sagemaker

account = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()

print(role)
print(bucket)

/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/site-packages/boto3/compat.py:82: PythonDeprecationWarning: Boto3 will no longer support Python 3.7 starting December 13, 2023. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.8 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


arn:aws:iam::687912291502:role/service-role/AmazonSageMaker-ExecutionRole-20211013T113123
sagemaker-us-west-2-687912291502


### baichuan fine tune 
1:使用pytorch ligtening框架  
2:deepspeed+QLoRA

In [21]:
#!rm -rf ./baichuan_finetuning
#!git clone https://github.com/ssbuild/baichuan_finetuning.git(弃用）
#!cp ./s5cmd ./baichuan_finetuning/
!rm -rf ./LLaMA-Efficient-Tuning
!git clone https://github.com/hiyouga/LLaMA-Efficient-Tuning.git
!cp ./s5cmd ./LLaMA-Efficient-Tuning/

Cloning into 'LLaMA-Efficient-Tuning'...
remote: Enumerating objects: 730, done.
remote: Counting objects: 100% (303/303), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 730 (delta 275), reused 259 (delta 258), pack-reused 427
Receiving objects: 100% (730/730), 72.12 MiB | 16.24 MiB/s, done.
Resolving deltas: 100% (515/515), done.
Updating files: 100% (47/47), done.


## prepare docker images

In [3]:
%%writefile Dockerfile
## You should change below region code to the region you used, here sample is use us-west-2
From 763104351884.dkr.ecr.us-west-2.amazonaws.com/huggingface-pytorch-training:1.13.1-transformers4.26.0-gpu-py39-cu117-ubuntu20.04 
#From pytorch/pytorch:1.5-cuda10.1-cudnn7-runtime

ENV LANG=C.UTF-8
ENV PYTHONUNBUFFERED=TRUE
ENV PYTHONDONTWRITEBYTECODE=TRUE

#RUN pip install -U git+https://github.com/ssbuild/deep_training.git


Overwriting Dockerfile


In [4]:
## You should change below region code to the region you used, here sample is use us-west-2
!aws ecr get-login-password --region us-west-2 | docker login --username AWS --password-stdin 763104351884.dkr.ecr.us-west-2.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


**Build image and push to ECR.**

In [8]:
## define repo name, should contain *sagemaker* in the name
repo_name = "sagemaker-baichuan_finetuning"

In [6]:
%%script env repo_name=$repo_name bash

#!/usr/bin/env bash

# This script shows how to build the Docker image and push it to ECR to be ready for use
# by SageMaker.

# The argument to this script is the image name. This will be used as the image on the local
# machine and combined with the account and region to form the repository name for ECR.
# The name of our algorithm
algorithm_name=${repo_name}

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

Login Succeeded
Sending build context to Docker daemon  29.04GB
Step 1/4 : From 763104351884.dkr.ecr.us-west-2.amazonaws.com/huggingface-pytorch-training:1.13.1-transformers4.26.0-gpu-py39-cu117-ubuntu20.04
 ---> c5a6ef695006
Step 2/4 : ENV LANG=C.UTF-8
 ---> Using cache
 ---> af49cfa7feae
Step 3/4 : ENV PYTHONUNBUFFERED=TRUE
 ---> Using cache
 ---> 287106637dc6
Step 4/4 : ENV PYTHONDONTWRITEBYTECODE=TRUE
 ---> Using cache
 ---> 773b4cf30c90
Successfully built 773b4cf30c90
Successfully tagged sagemaker-baichuan_finetuning:latest
The push refers to repository [687912291502.dkr.ecr.us-west-2.amazonaws.com/sagemaker-baichuan_finetuning]
f8dae5c3df1e: Preparing
e3221f18601a: Preparing
b6f286626882: Preparing
76fe97d80cdb: Preparing
f5f76489fff8: Preparing
621c3f07daa7: Preparing
9b484bb42e11: Preparing
54c7c0b58471: Preparing
c34adc3ab668: Preparing
bbf651e48b84: Preparing
f61045791108: Preparing
4e2ac0cda74a: Preparing
658a33d555eb: Preparing
bd16d9a61a98: Preparing
f0c0cd2accfa: Preparin

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



## Train ligtning baichuan!

In [11]:
%%writefile ./baichuan_finetuning/config/main.py


# -*- coding: utf-8 -*-
# @Author  : ssbuild
# @Time    : 2023/5/31 14:43

import json
import os
import torch
import torchvision
torch.__version__
torchvision.__version__

train_info_args = {
    'devices': 4,
}

global_args = {
          "load_in_8bit": False, # lora 如果显卡支持int8 可以开启 ， 需安装依赖 pip install bitsandbytes
          "num_layers": -1, # 是否使用骨干网络的全部层数 ， -1 表示全层, 否则只用只用N层
          "num_layers_key":  "num_layers"
}

lora_info_args = {
               'with_lora': True,  # 是否启用lora模块
               'r': 8,
               'target_modules': ['query_key_value'],
               'target_dtype': None,
               'lora_alpha': 32,
               'lora_dropout': 0.1,
               'bias': 'none',  # Bias type for Lora. Can be 'none', 'all' or 'lora_only'"
               'modules_to_save' : None, # "help": "List of modules apart from LoRA layers to be set as trainable and saved in the final checkpoint. "
}


# 模块配置， 默认启用lora
enable_deepspeed = True
enable_ptv2 = False
enable_lora = True
load_in_bit = 0  # 4 load_in_4bit, 8 load_in_8bit  other  0



if enable_lora:
    from config.sft_config_lora import *
elif enable_ptv2:
    from config.sft_config_ptv2 import *
else:
    from config.sft_config import *



if enable_lora:
    enable_ptv2 = False
    global_args['load_in_4bit'] = load_in_bit == 4
    global_args['load_in_8bit'] = load_in_bit == 8

    if global_args['load_in_4bit']:
        global_args['quantization_config'] = None

    #检查lora adalora是否开启
    if 'lora' not in train_info_args and 'adalora' not in train_info_args:
        raise ValueError('please config lora or adalora')
    if train_info_args.get('lora',{}).get('with_lora',False) and train_info_args.get('adalora',{}).get('with_lora',False):
        raise Exception('lora and adalora can set one at same time !')

    train_info_args.pop('prompt', None)
elif enable_ptv2:
    enable_lora = False
    global_args['load_in_4bit'] = False
    global_args['load_in_8bit'] = False
    train_info_args.pop('lora', None)
    train_info_args.pop('adalora', None)
else:
    # global_args['load_in_4bit'] = False
    # global_args['load_in_8bit'] = False
    train_info_args.pop('lora',None)
    train_info_args.pop('adalora', None)
    train_info_args.pop('prompt', None)

#预处理
if 'rwkv' in train_info_args['tokenizer_name'].lower():
    train_info_args['use_fast_tokenizer'] = True


def get_deepspeed_config():
    '''
        lora prompt finetuning 使用 deepspeed_offload.json
        普通finetuning 使用deepspeed.json
    '''
    # 是否开启deepspeed
    if not enable_deepspeed:
        return None

    # 选择 deepspeed 配置文件
    is_need_update_config = False
    if enable_lora:
        is_need_update_config = True
        filename = os.path.join(os.path.dirname(__file__), 'deepspeed_offload.json')
    else:
        filename = os.path.join(os.path.dirname(__file__), 'deepspeed.json')


    with open(filename, mode='r', encoding='utf-8') as f:
        deepspeed_config = json.loads(f.read())

    #lora offload 同步优化器配置
    if is_need_update_config:
        optimizer = deepspeed_config.get('optimizer',None)
        if optimizer:
            optimizer['params']['betas'] = train_info_args.get('optimizer_betas', (0.9, 0.999))
            optimizer['params']['lr'] = train_info_args.get('learning_rate', 2e-5)
            optimizer['params']['eps'] = train_info_args.get('adam_epsilon', 1e-8)
    return deepspeed_config


Overwriting ./baichuan_finetuning/config/main.py


In [12]:
%%writefile ./baichuan_finetuning/train.sh
#!/bin/bash

chmod +x ./s5cmd
pip install -U -r requirements.txt

#制作数据
python data_utils.py
    
#训练
python train.py

./s5cmd sync ./best_ckpt s3://$MODEL_S3_BUCKET/models/baichuan_finetuning/output/$(date +%Y-%m-%d-%H-%M-%S)/

Overwriting ./baichuan_finetuning/train.sh


In [7]:
## The image uri which is build and pushed above
image_uri = "{}.dkr.ecr.{}.amazonaws.com/{}:latest".format(account, region, repo_name)
image_uri

'687912291502.dkr.ecr.us-west-2.amazonaws.com/sagemaker-baichuan_finetuning:latest'

In [ ]:
import time
from sagemaker.estimator import Estimator

environment = {
              'MODEL_S3_BUCKET': bucket # The bucket to store pretrained model and fine-tune model
}

base_job_name = 'baichuan-finetuning'

instance_type = 'ml.g5.12xlarge'

estimator = Estimator(role=role,
                      entry_point='train.sh',
                      source_dir='./baichuan_finetuning/',
                      base_job_name=base_job_name,
                      instance_count=1,
                      instance_type=instance_type,
                      image_uri=image_uri,
                      environment=environment,
                      disable_profiler=True,
                      debugger_hook_config=False,
                      max_run=24*60*60*2)

estimator.fit()

## train GPT4 baichuan

In [ ]:
##基础模型
!aws s3 ls s3://sagemaker-us-west-2-687912291502/llm/models/LLM_baichuan_model/

In [ ]:
## 合并基础模型及LoRA权重
#python src/export_model.py --checkpoint_dir path_to_checkpoint --output_dir path_to_save_model --model_name_or_path model_name_or_path

In [12]:
####下载的baichuan model s3路径 ###########
model_s3_path="s3://sagemaker-us-west-2-687912291502/llm/models/LLM_baichuan_model"

In [13]:
## The image uri which is build and pushed above
image_uri = "{}.dkr.ecr.{}.amazonaws.com/{}:latest".format(account, region, repo_name)
image_uri

'687912291502.dkr.ecr.us-west-2.amazonaws.com/sagemaker-baichuan_finetuning:latest'

In [14]:
%%writefile ./LLaMA-Efficient-Tuning/requirements.txt
transformers>=4.29.1
datasets>=2.12.0
accelerate>=0.19.0
peft>=0.3.0
trl>=0.4.4
sentencepiece
jieba
rouge-chinese
nltk
gradio
mdtex2html
uvicorn
fastapi
sse-starlette

Overwriting ./LLaMA-Efficient-Tuning/requirements.txt


In [19]:
%%writefile ./LLaMA-Efficient-Tuning/train.sh
#!/bin/bash

chmod +x ./s5cmd
pip install -U -r requirements.txt

#./s5cmd sync $MODEL_S3_PATH/* /tmp/baichun-7b/


export CUDA_VISIBLE_DEVICES=0
python src/train_sft.py \
    --model_name_or_path "fireballoon/baichuan-vicuna-7b" \
    --do_train \
    --dataset alpaca_zh \
    --finetuning_type lora \
    --output_dir /tmp/ouput/ \
    --overwrite_cache \
    --per_device_train_batch_size 4 \
    --gradient_accumulation_steps 4 \
    --lr_scheduler_type cosine \
    --logging_steps 10 \
    --save_steps 1000 \
    --learning_rate 5e-5 \
    --num_train_epochs 1.0 \
    --plot_loss \
    --fp16

./s5cmd sync /tmp/ouput/ $MODEL_S3_PATH/models/baichuan_finetuning/output/$(date +%Y-%m-%d-%H-%M-%S)/

Overwriting ./LLaMA-Efficient-Tuning/train.sh


In [20]:
import time
from sagemaker.estimator import Estimator

environment = {
              'MODEL_S3_PATH': model_s3_path # The bucket to store pretrained model and fine-tune model
}

base_job_name = 'baichuan-finetuning'

instance_type = 'ml.g5.4xlarge'

estimator = Estimator(role=role,
                      entry_point='train.sh',
                      source_dir='./LLaMA-Efficient-Tuning/',
                      base_job_name=base_job_name,
                      instance_count=1,
                      instance_type=instance_type,
                      image_uri=image_uri,
                      environment=environment,
                      disable_profiler=True,
                      debugger_hook_config=False,
                      max_run=24*60*60*2)

estimator.fit()

Using provided s3_resource


INFO:sagemaker:Creating training-job with name: baichuan-finetuning-2023-07-10-14-09-07-169


2023-07-10 14:09:29 Starting - Starting the training job...
2023-07-10 14:09:45 Starting - Preparing the instances for training......
2023-07-10 14:10:52 Downloading - Downloading input data...
2023-07-10 14:11:07 Training - Downloading the training image........................
2023-07-10 14:15:13 Training - Training image download completed. Training in progress.....bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-07-10 14:16:01,032 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-07-10 14:16:01,048 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-07-10 14:16:01,058 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-07-10 14:16:01,060 sagemaker_pytorch_container.training INFO     Invoking user training script.
2023-07-10 14:16:04,047 sagemaker-training-toolkit INFO     No Neurons dete